<div class="alert alert-block alert-info"><b>Optimisation with JuMP:</b> This notebook contains three examples of textbook/toy optimisation problems solved with Julia's <a href="http://www.juliaopt.org/JuMP.jl/v0.21/">JuMP domain-specific modelling language</a>. These are a <b>linear integer programming problem</b> (choosing the best mix of promotional channels), a <b>linear assignment problem</b> (assigning operators to machines), and a <b>non-linear optimisation problem</b> (maximising revenue subject to constraints).</div>

## Integer programming
Problem from Bradley et al, Applied Mathematical Programming (p.305). A firm (in the 1970s) wants to optimise its marketing spend to reach the maximum audience. The options and constraints are:

| Attribute \ Medium | TV | Trade Magazine | Newspaper | Radio | Popular Magazine | Promo | Total Resource Available |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Customers reached | 1,000,000 | 200,000 | 300,000 | 400,000 | 450,000 | 450,000 |  |
| Cost ($) | 500,000 | 150,000 | 300,000 | 250,000 | 250,000 | 100,000 | 1,800,000 |
| Designers needed (person-hours) | 700 | 250 | 200 | 200 | 300 | 400 | 1,500 |
| Salespeople needed (person-hours) | 200 | 100 | 100 | 100 | 100 | 1000 | 1,200 |

In addition, the following constraints have to be met:

- If the promotional campaign is undertaken, it needs either a radio or a popular magazine campaign effort to support it.
- The firm cannot advertise in both the trade and popular magazines.


In [ ]:
using JuMP, GLPK
m1 = Model(with_optimizer(GLPK.Optimizer))

costs_matrix = [5e5 1.5e5 3e5 2.5e5 2.5e5 1e5; 700 250 200 200 300 400; 200 100 100 100 100 1000]
cons_vec = [1.8e6, 1500, 1200]

@variable(m1, x[1:6], Bin)

@objective(m1, Max, 1000000x[1] + 200000x[2] + 300000x[3] + 400000x[4] + 450000x[5] + 450000x[6])

@constraint(m1, cons, costs_matrix * x .<= cons_vec)
@constraint(m1, x[6] <= x[4] + x[5]) # promo needs radio or popular mags
@constraint(m1, x[2] + x[5] <= 1) # trade xor popular mags

# 1 tv
# 2 trade_mag
# 3 newspaper
# 4 radio
# 5 pop_mag
# 6 promo

Solve the model and report its status:

In [ ]:
optimize!(m1)
termination_status(m1)

In [ ]:
value.(x)

In [ ]:
objective_value(m1)

## An assignment problem
Problem from Fitzharris, Linear Optimisation with Applications (p.212). Consider the problem of assigning four machine operators exclusively to four machines. The costs of assigning each operator to each machine are tabulated below. Dave is not qualified on the miller, and Ben has not been trained on the drill.

| Operator \ Machine | Grinder | Lathe | Miller | Drill | 
| --- | --- | --- | --- | --- | 
| Dave | 5 | 5 | - | 2 |
| Sue | 7 | 4 | 2 | 3 |
| Ben | 9 | 3 | 5 | - |
| Sandeep | 7 | 2 | 6 | 7 |

Which assignment minimises the total cost, and what is that cost?

In [ ]:
using JuMP, GLPK
m2 = Model(with_optimizer(GLPK.Optimizer))

costs_matrix = [5 5 0 2; 7 4 2 3; 9 3 5 0; 7 2 6 7]

@variable(m2, x[1:4, 1:4], Bin)
@objective(m2, Min, sum(costs_matrix.*x))
@constraint(m2, sum(x, dims = 1) .== 1) # sums over each col
@constraint(m2, sum(x, dims = 2) .== 1) # sums over each row
@constraint(m2, x[1,3] == 0)
@constraint(m2, x[3,4] == 0)

In [ ]:
optimize!(m2)
termination_status(m2)

The status code tells us the model has found an optimal solution. Let's see what it is:

In [ ]:
value.(x)

In [ ]:
objective_value(m2)

So the minimal cost is 14 units, when Dave is assigned to the drill, Sue is assigned to the miller, Ben is assigned to the lathe, and Sandeep is assigned to the grinder.  

## Non-linear optimisation

Problem from Lambert, Advanced Mathematics for Economists (p.130). Suppose a firm has total revenue $TR = 10Q - Q^2 + A/2$ where $Q$ is its output and $A$ is its advertising expenditure. Its total costs are $TC = Q^2/2 + 5Q + 1 + A$. The management seek to maximise revenue subject to the minimum profit constraint $\pi \geq \pi_0$, with $Q \geq 0$ and $A \geq 0$, in the case where $\pi_o = 1/2$.

Combining the expressions for $TR$, $TC$, and $\pi_o$ into a single non-linear constraint, we have:

In [ ]:
using JuMP, Ipopt
m3 = Model(with_optimizer(Ipopt.Optimizer))
@variable(m3, Q >= 0, start = 0.0,)
@variable(m3, A >=0, start = 0.0)
π_0 = 1/2
@NLobjective(m3, Max, 10Q - Q^2 + A/2)
@NLconstraint(m3, 3Q^2/2 - 5Q + A/2 + 1 + π_0 <= 0)

Let's solve the model (this will produce a long section of output!):

In [45]:
optimize!(m3)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

Let's see the results:

In [46]:
objective_value(m3)

21.000000008161187

So the maximum revenue is 21 units, obtained when:

In [47]:
value.([Q, A])

2-element Array{Float64,1}:
 2.9999743655103503    
 0.00020509355382131316

$Q = 3$ and $A = 0$ (to 3 significant figures). This agrees with the analytical solution obtained in the source text.

## References

Bradley, SP, Hax, AC, and Magnanti, TL (1977). Applied Mathematical Programming. Addison Wesley: Reading (MA), USA.

Fitzharris, AM (2019). Linear Optimisation with Applications. Grosvenor House Publishing: Tolworth, UK.

Lambert, PJ (1985). Advanced Mathematics for Economists: Static and Dynamic Optimization. Blackwell: Oxford, UK.